# 1. Import Dependencies

In [ ]:
!pip install opencv-python

In [ ]:
# Import opencv
import cv2
# Import Operating System
import os
# Import Random
import random
# Import Numpy
import numpy as np

# 2. Define Images to Collect

In [ ]:
labels = ['b01-000', 'b02-000', 'b03-000', 'c01-000', 'c06-000']
number_imgs = 5

# 3. Setup Folders

In [ ]:
ipynb_path = os.path.dirname(os.path.realpath("__file__"))
#print(ipynb_path)
root = "\\".join(ipynb_path.split("\\")[:-2])
#print(root)
ORIGINALS_PATH = os.path.join(root, 'datasets', 'originals')
original_folders = ['backgrounds', 'overlays']
ROOT_PATH = os.path.join(root, 'datasets', 'tftest')
IMAGES_PATH = os.path.join(ROOT_PATH, 'images')
images_folders = ['train', 'test']

In [ ]:
if not os.path.exists(ORIGINALS_PATH):
    if os.name == 'posix':
        !mkdir -p {ORIGINALS_PATH}
    if os.name == 'nt':
         !mkdir {ORIGINALS_PATH}
for folder in original_folders:
    path = os.path.join(ORIGINALS_PATH, folder)
    if not os.path.exists(path):
        !mkdir {path}

if not os.path.exists(ROOT_PATH):
    if os.name == 'posix':
        !mkdir -p {ROOT_PATH}
    if os.name == 'nt':
         !mkdir {ROOT_PATH}
if not os.path.exists(IMAGES_PATH):
    !mkdir {IMAGES_PATH}
for folder in images_folders:
    path = os.path.join(IMAGES_PATH, folder)
    if not os.path.exists(path):
        !mkdir {path}

# 4. Create Images & Annoations

In [ ]:
BACKGROUNDS_PATH = os.path.join(ORIGINALS_PATH, 'backgrounds')
_, _, background_files = next(os.walk(BACKGROUNDS_PATH), (None, [], []))
print("background files: " + str(len(background_files)))

OVERLAYS_PATH = os.path.join(ORIGINALS_PATH, 'overlays')
_, overlay_folders, _ = next(os.walk(OVERLAYS_PATH), (None, [], []))
print("overlay folders: " + str(len(overlay_folders)))

all_overlay_files = []

for (m, overlay_folder) in enumerate(overlay_folders):
    path = os.path.join(OVERLAYS_PATH, overlay_folder)
    _, _, overlay_files = next(os.walk(path), (None, [], []))
    for (d, overlay_file) in enumerate(overlay_files):
        #print("overlay file: " + overlay_file)
        overlay_path = os.path.join(path, overlay_file)
        all_overlay_files.append(overlay_path)

print("all overlay files: " + str(len(all_overlay_files)))
#print(all_overlay_files)

In [ ]:
for (o, overlay_path) in enumerate(all_overlay_files):

    _, class_name, file = overlay_path.split("\\")[-3:]
    overlay_name, _ = file.split(".")[-2:]
    
    overlay = cv2.imread(overlay_path, cv2.IMREAD_UNCHANGED)

    y, x = overlay[:, :, 3].nonzero()
    minx = np.min(x)
    miny = np.min(y)
    maxx = np.max(x)
    maxy = np.max(y)
    overlay = overlay[miny:maxy, minx:maxx]
    
    overlay_height, overlay_width = overlay.shape[:2]
    
    for i in range(number_imgs+1):
        
        targetName = class_name+"-"+overlay_name+"-"+str(i)+".png"
        aTargetName = class_name+"-"+overlay_name+"-"+str(i)+".xml"
        tot = 'train'
        if i==number_imgs:
            tot = 'test'
        
        background_file = random.choice(background_files)
        background_path = os.path.join(BACKGROUNDS_PATH, background_file)
        background = cv2.imread(background_path)
        background_height, background_width = background.shape[:2]
        dim = (background_height, background_width)
        if background_height < overlay_height:
            dim[0] = overlay_height
        if background_width < overlay_width:
            dim[1] = overlay_width
        if dim != (background_height, background_width):
            background = cv2.resize(background, dim, interpolation=cv2.INTER_AREA)

        rndy = 0
        if dim[0] > overlay_height:
            rndy = random.randint(0, dim[0]-overlay_height)

        rndx = 0
        if dim[1] > overlay_width:
            rndx = random.randint(0, dim[1]-overlay_width)
        
        combine_image = background
        for y in range(overlay_height):
            for x in range(overlay_width):
                overlay_color = overlay[y, x, :3]
                overlay_alpha = overlay[y, x, 3] / 255
                background_color = background[rndy+y, rndx+x]
                combine_color = background_color*(1-overlay_alpha)+overlay_color*overlay_alpha
                combine_image[rndy+y, rndx+x] = combine_color
        
        target_path = os.path.join(IMAGES_PATH, tot, targetName)
        a_target_path = os.path.join(IMAGES_PATH, tot, aTargetName)
        print(target_path)
        cv2.imwrite(target_path, combine_image)
        anotation = "<annotation>\n"+ \
        "\t<folder>"+tot+"</folder>\n"+ \
        "\t<filename>"+targetName+"</filename>\n"+ \
        "\t<path>"+root+"\\datasets\\tftest\images\\"+tot+"\\"+targetName+"</path>\n"+ \
        "\t<source>\n"+ \
        "\t\t<database>Unknown</database>\n"+ \
        "\t</source>\n"+ \
        "\t<size>\n"+ \
        "\t\t<width>"+str(dim[1])+"</width>\n"+ \
        "\t\t<height>"+str(dim[0])+"</height>\n"+ \
        "\t\t<depth>3</depth>\n"+ \
        "\t</size>\n"+ \
        "\t<segmented>0</segmented>\n"+ \
        "\t<object>\n"+ \
        "\t\t<name>"+class_name+"</name>\n"+ \
        "\t\t<pose>Unspecified</pose>\n"+ \
        "\t\t<truncated>0</truncated>\n"+ \
        "\t\t<difficult>0</difficult>\n"+ \
        "\t\t<bndbox>\n"+ \
        "\t\t\t<xmin>"+str(rndx)+"</xmin>\n"+ \
        "\t\t\t<ymin>"+str(rndy)+"</ymin>\n"+ \
        "\t\t\t<xmax>"+str(rndx+overlay_width)+"</xmax>\n"+ \
        "\t\t\t<ymax>"+str(rndy+overlay_height)+"</ymax>\n"+ \
        "\t\t</bndbox>\n"+ \
        "\t</object>\n"+ \
        "</annotation>"
        f = open(a_target_path, "w")
        f.write(anotation)
        f.close()